### Imports

In [1]:
import pandas as pd
import numpy as np
import joblib
import tkinter as tk
from tkinter import messagebox
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

C:\Users\Ali\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.head(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [5]:
df.shape

(10886, 12)

In [6]:
df.info

<bound method DataFrame.info of                   datetime  season  holiday  workingday  weather   temp  \
0      2011-01-01 00:00:00       1        0           0        1   9.84   
1      2011-01-01 01:00:00       1        0           0        1   9.02   
2      2011-01-01 02:00:00       1        0           0        1   9.02   
3      2011-01-01 03:00:00       1        0           0        1   9.84   
4      2011-01-01 04:00:00       1        0           0        1   9.84   
...                    ...     ...      ...         ...      ...    ...   
10881  2012-12-19 19:00:00       4        0           1        1  15.58   
10882  2012-12-19 20:00:00       4        0           1        1  14.76   
10883  2012-12-19 21:00:00       4        0           1        1  13.94   
10884  2012-12-19 22:00:00       4        0           1        1  13.94   
10885  2012-12-19 23:00:00       4        0           1        1  13.12   

        atemp  humidity  windspeed  casual  registered  count  
0  

### Feature Engineering

In [7]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month
df['weekday'] = df['datetime'].dt.weekday
df.drop(columns=['datetime','casual','registered'], inplace=True)

### Define X and y

In [8]:
X = df.drop(columns=['count'])
y = df['count']

In [9]:
X

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,hour,day,month,weekday
0,1,0,0,1,9.84,14.395,81,0.0000,0,1,1,5
1,1,0,0,1,9.02,13.635,80,0.0000,1,1,1,5
2,1,0,0,1,9.02,13.635,80,0.0000,2,1,1,5
3,1,0,0,1,9.84,14.395,75,0.0000,3,1,1,5
4,1,0,0,1,9.84,14.395,75,0.0000,4,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,19,19,12,2
10882,4,0,1,1,14.76,17.425,57,15.0013,20,19,12,2
10883,4,0,1,1,13.94,15.910,61,15.0013,21,19,12,2
10884,4,0,1,1,13.94,17.425,61,6.0032,22,19,12,2


In [10]:
y

0         16
1         40
2         32
3         13
4          1
        ... 
10881    336
10882    241
10883    168
10884    129
10885     88
Name: count, Length: 10886, dtype: int64

### Preprocessing Pipeline

In [11]:
cat_features = ['season','holiday','workingday','weather']
num_features = [c for c in X.columns if c not in cat_features]

num_pipeline = Pipeline([('scaler', StandardScaler())])
cat_pipeline = Pipeline([('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

### Train / Validation Split

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### Apply Preprocessing

In [13]:
X_train_p = preprocessor.fit_transform(X_train)
X_val_p = preprocessor.transform(X_val)

### Build ANN Model

In [14]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_p.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

C:\Users\Ali\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Train Model

In [15]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train_p, y_train, validation_data=(X_val_p, y_val), epochs=100, batch_size=64, callbacks=[early_stop], verbose=1)

Epoch 1/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 41486.6172 - mae: 143.2585 - val_loss: 21414.3027 - val_mae: 106.6127
Epoch 2/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 21497.3809 - mae: 107.1675 - val_loss: 20828.0098 - val_mae: 103.9783
Epoch 3/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 21032.0332 - mae: 104.9978 - val_loss: 20593.3066 - val_mae: 101.4846
Epoch 4/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 20764.6094 - mae: 103.7832 - val_loss: 20194.2461 - val_mae: 101.9155
Epoch 5/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 20331.0508 - mae: 102.6900 - val_loss: 19744.0684 - val_mae: 99.4134
Epoch 6/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19938.8711 - mae: 101.4606 - val_loss: 19222.0293 - val_mae: 97.5126
Epoch 7/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 19366.5293 - mae: 98.9840 - val_loss: 18307.5723 - val_mae: 96.6902
Epoch 8/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 18631.9668 - mae: 96.8946 

### Evaluate Model

In [16]:
pred = model.predict(X_val_p).flatten()
rmse = np.sqrt(mean_squared_error(y_val, pred))
mae = mean_absolute_error(y_val, pred)
r2 = r2_score(y_val, pred)
print(f"RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.4f}")

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
RMSE: 70.16, MAE: 48.85, R²: 0.8509


### Save Model & Preprocessor

In [17]:
model.save("model.h5")
joblib.dump(preprocessor, "preprocessor.pkl")

['preprocessor.pkl']

### GUI with Tkinter

In [18]:
def predict():
    try:
        data = {
            "season": int(season_var.get()),
            "holiday": int(holiday_var.get()),
            "workingday": int(workingday_var.get()),
            "weather": int(weather_var.get()),
            "temp": float(temp_var.get()),
            "atemp": float(atemp_var.get()),
            "humidity": int(humidity_var.get()),
            "windspeed": float(windspeed_var.get()),
            "hour": int(hour_var.get()),
            "day": int(day_var.get()),
            "month": int(month_var.get()),
            "weekday": int(weekday_var.get())
        }
        df_input = pd.DataFrame([data])
        X_input = preprocessor.transform(df_input)
        pred_val = model.predict(X_input)[0][0]
        result_label.config(text=f"🚲 Predicted Demand: {int(pred_val)} bikes")
    except Exception as e:
        messagebox.showerror("Error", str(e))

root = tk.Tk()
root.title("Bike Demand Prediction - ANN")
root.geometry("420x600")
root.resizable(False, False)

tk.Label(root, text="🚲 Bike Sharing Demand Prediction", font=("Arial", 14, "bold")).pack(pady=10)

frame = tk.Frame(root)
frame.pack(pady=10)

fields = [
    ("Hour", 0, 23),
    ("Day", 1, 31),
    ("Month", 1, 12),
    ("Weekday (0=Mon)", 0, 6),
    ("Season (1-4)", 1, 4),
    ("Holiday (0/1)", 0, 1),
    ("Working Day (0/1)", 0, 1),
    ("Weather (1-4)", 1, 4),
    ("Temperature", 0, 40),
    ("Feels Like Temp", 0, 50),
    ("Humidity", 0, 100),
    ("Windspeed", 0, 50)
]

vars_list = []
for i, (label, minv, maxv) in enumerate(fields):
    tk.Label(frame, text=label).grid(row=i, column=0, sticky="w", pady=3)
    var = tk.StringVar()
    tk.Entry(frame, textvariable=var).grid(row=i, column=1, pady=3)
    vars_list.append(var)

(hour_var, day_var, month_var, weekday_var,
 season_var, holiday_var, workingday_var, weather_var,
 temp_var, atemp_var, humidity_var, windspeed_var) = vars_list

tk.Button(root, text="🚀 Predict", font=("Arial", 12, "bold"),
          bg="#4CAF50", fg="white", width=20, command=predict).pack(pady=15)

result_label = tk.Label(root, text="Prediction: ---", font=("Arial", 13, "bold"))
result_label.pack(pady=10)

root.mainloop()